# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 12 - Processamento de Linguagem Natural</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.8.2


Obs: Este é um material de bônus incluído neste curso. PyTorch é estudado em detalhes no curso <a href="https://www.datascienceacademy.com.br/course?courseid=deep-learning-frameworks">Deep Learning Frameworks</a> e aplicado em PLN no curso <a href="https://www.datascienceacademy.com.br/course?courseid=processamento-de-linguagem-natural-e-reconhecimento-de-voz">Processamento de Linguagem Natural</a>.

## Modelo de Classificação de Idiomas de Sentenças com Bag of Words e PyTorch

![title](imagens/bow.png)

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
# Instala o PyTorch
#!pip install -q -U torch torchvision

In [4]:
# Imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch import nn, optim

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

numpy     : 1.19.2
matplotlib: 3.5.0
torch     : 1.11.0
pandas    : 1.3.5



### Preparando os Dados

In [6]:
# Dados de treino
dados_treino = [("Tenho vinte paginas de leitura".lower().split(), "Portuguese"),
                ("I will visit the library".lower().split(), "English"),
                ("I am reading a book".lower().split(), "English"),
                ("This is my favourite chapter".lower().split(), "English"),
                ("Estou na biblioteca lendo meu livro preferido".lower().split(), "Portuguese"),
                ("Gosto de livros sobre viagens".lower().split(), "Portuguese")]

In [7]:
dados_treino

[(['tenho', 'vinte', 'paginas', 'de', 'leitura'], 'Portuguese'),
 (['i', 'will', 'visit', 'the', 'library'], 'English'),
 (['i', 'am', 'reading', 'a', 'book'], 'English'),
 (['this', 'is', 'my', 'favourite', 'chapter'], 'English'),
 (['estou', 'na', 'biblioteca', 'lendo', 'meu', 'livro', 'preferido'],
  'Portuguese'),
 (['gosto', 'de', 'livros', 'sobre', 'viagens'], 'Portuguese')]

In [8]:
# Dados de teste
dados_teste = [("Estou lendo".lower().split(), "Portuguese"),
               ("This is not my favourite book".lower().split(), "English")]

In [9]:
dados_teste

[(['estou', 'lendo'], 'Portuguese'),
 (['this', 'is', 'not', 'my', 'favourite', 'book'], 'English')]

In [65]:
# Prepara o dicionário do vocabulário

# Dicionário para o vocabulário
dict_vocab = {}

# Contadoor
i = 0

# Loop pelos dados de treino e teste
for palavras, idiomas in dados_treino + dados_teste:
    for palavra in palavras:
        if palavra not in dict_vocab:
            dict_vocab[palavra] = i
            i += 1

In [66]:
# Visualiza o vocabulário
print(dict_vocab)

{'tenho': 0, 'vinte': 1, 'paginas': 2, 'de': 3, 'leitura': 4, 'i': 5, 'will': 6, 'visit': 7, 'the': 8, 'library': 9, 'am': 10, 'reading': 11, 'a': 12, 'book': 13, 'this': 14, 'is': 15, 'my': 16, 'favourite': 17, 'chapter': 18, 'estou': 19, 'na': 20, 'biblioteca': 21, 'lendo': 22, 'meu': 23, 'livro': 24, 'preferido': 25, 'gosto': 26, 'livros': 27, 'sobre': 28, 'viagens': 29, 'not': 30}


In [67]:
# Tamanho do corpus
tamanho_corpus = len(dict_vocab)

In [68]:
tamanho_corpus

31

In [14]:
# Número de idiomas
idiomas = 2

In [15]:
# Índice para os idiomas
label_index = {"Portuguese": 0, "English": 1}

### Construção do Modelo

In [16]:
# Classe para o modelo BOW de classificação
class ModeloBOW(nn.Module):  

    # Método construtor
    def __init__(self, lista_idiomas, tamanho_do_corpus):
        super(ModeloBOW, self).__init__()
        self.linear = nn.Linear(tamanho_do_corpus, lista_idiomas)

    # Feed Forward
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim = 1)

In [17]:
# Função para criar o vetor BOW necessário para o treinamento
def cria_bow_vetor(sentence, word_index):
    word_vec = torch.zeros(tamanho_corpus)
    for word in sentence:
        # Adicionar palavra ao dicionário e aumentar o tamanho corpus
        word_vec[dict_vocab[word]] += 1
    return word_vec.view(1, -1)

In [18]:
# Função para criar a variável target
def cria_target(label, label_index):
    return torch.LongTensor([label_index[label]])

In [19]:
# Cria o modelo
modelo = ModeloBOW(idiomas, tamanho_corpus)

In [20]:
# Função de perda (loss)
loss_function = nn.NLLLoss()

In [21]:
# Otimizador
optimizer = optim.SGD(modelo.parameters(), lr = 0.1)

### Treinamento do Modelo

In [22]:
# Loop de treinamentoo
for epoch in range(100):
    
    for sentence, label in dados_treino:

        modelo.zero_grad()

        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        target = cria_target(label, label_index)

        log_probs = modelo(bow_vec)

        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print('Epoch: ', str(epoch+1),', Loss: ' + str(loss.item()))

Epoch:  1 , Loss: 1.0166406631469727
Epoch:  11 , Loss: 0.0929742231965065
Epoch:  21 , Loss: 0.04588890075683594
Epoch:  31 , Loss: 0.030261052772402763
Epoch:  41 , Loss: 0.022522488608956337
Epoch:  51 , Loss: 0.017916858196258545
Epoch:  61 , Loss: 0.01486651785671711
Epoch:  71 , Loss: 0.012699330225586891
Epoch:  81 , Loss: 0.011080989614129066
Epoch:  91 , Loss: 0.00982685573399067


### Previsões e Avaliação do Modelo

In [52]:
def validate_news_words(sentence):
    for word in sentence:
        if dict_vocab.get(word) == None:
            print("\t adicionando:", word)
            dict_vocab[word] = len(dict_vocab)
            print(dict_vocab)
            tamanho_corpus = len(dict_vocab)
            print(tamanho_corpus)

In [53]:
# Função para previsões
def faz_previsao(data):

    with torch.no_grad():
        sentence = data[0]
        label = data[1]
        validate_news_words(sentence)
        bow_vec = cria_bow_vetor(sentence, dict_vocab)
        log_probs = modelo(bow_vec)
        print(sentence)
        print('Probabilidade de ser o label: ' + label, 'é igual a: ',  np.exp(log_probs))

In [54]:
# Previsão com a primeira sentença de teste
faz_previsao(dados_teste[0])

['estou', 'lendo']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.7613, 0.2387]])


In [55]:
dados_teste[0]

(['estou', 'lendo'], 'Portuguese')

In [56]:
# Previsão com a segunda sentença de teste
faz_previsao(dados_teste[1])

['this', 'is', 'not', 'my', 'favourite', 'book']
Probabilidade de ser o label: English é igual a:  tensor([[0.0123, 0.9877]])


In [57]:
dados_teste[1]

(['this', 'is', 'not', 'my', 'favourite', 'book'], 'English')

### Previsões com Novas Frases

In [58]:
# Nova frase
novas_frases = [("Tenho livros sobre viagens".lower().split(), "Portuguese"),
                ("Estou escrevendo".lower().split(), "Portuguese"),
                ("Gosto de biblioteca".lower().split(), "Portuguese"),
                ("Eu perdi o meu mouse".lower().split(), "Portuguese")]

In [59]:
novas_frases

[(['tenho', 'livros', 'sobre', 'viagens'], 'Portuguese'),
 (['estou', 'escrevendo'], 'Portuguese'),
 (['gosto', 'de', 'biblioteca'], 'Portuguese'),
 (['eu', 'perdi', 'o', 'meu', 'mouse'], 'Portuguese')]

In [60]:
faz_previsao(novas_frases[0])

['tenho', 'livros', 'sobre', 'viagens']
Probabilidade de ser o label: Portuguese é igual a:  tensor([[0.9679, 0.0321]])


In [69]:
faz_previsao(novas_frases[1])

	 adicionando: escrevendo
{'tenho': 0, 'vinte': 1, 'paginas': 2, 'de': 3, 'leitura': 4, 'i': 5, 'will': 6, 'visit': 7, 'the': 8, 'library': 9, 'am': 10, 'reading': 11, 'a': 12, 'book': 13, 'this': 14, 'is': 15, 'my': 16, 'favourite': 17, 'chapter': 18, 'estou': 19, 'na': 20, 'biblioteca': 21, 'lendo': 22, 'meu': 23, 'livro': 24, 'preferido': 25, 'gosto': 26, 'livros': 27, 'sobre': 28, 'viagens': 29, 'not': 30, 'escrevendo': 31}
32


IndexError: index 31 is out of bounds for dimension 0 with size 31

In [ ]:
faz_previsao(novas_frases[2])

In [ ]:
faz_previsao(novas_frases[3])

# Fim